In [1]:
#Use kernel spyglass-dlc, NOT spyglass
%run common_imports.py
import spyglass.position.v1 as sgp
import pynwb
import yaml
from spyglass.position import PositionOutput
from pathlib import Path

dj.config['display.limit'] = 10**3
%matplotlib qt
plt.style.use(["seaborn-v0_8-talk"])
%load_ext autoreload
%autoreload 2

[2025-12-16 15:37:56,421][INFO]: Connecting anirudh@172.16.102.154:3306
[2025-12-16 15:37:56,438][INFO]: Connected anirudh@172.16.102.154:3306


### Populate sgp.DLCModel() computed table

#### Insert into DLCProject()

In [5]:
animal = "wilbur"
dlc_project_dir = Path(f"/media/labuser/NA_1_2025/spyglass/{animal}/position_models/WtrackSep5-trainset95shuffle1/")
project_name = f"{animal}-WtrackSep5-trainset95shuffle1"
model_name = "WtrackSep5-trainset95shuffle1"
scorer = "TM"

config_path = dlc_project_dir / "config.yaml"
assert config_path.exists(), f"config.yaml not found at {config_path}"


In [14]:
sgp.DLCProject()

project_name name of DLC project,team_name,bodyparts list of bodyparts to label,frames_per_video number of frames to extract from each video,config_path path to config.yaml for model
wilbur-WtrackSep5-trainset95shuffle1,NeuroACT,=BLOB=,20,/media/labuser/NA_1_2025/spyglass/wilbur/position_models/WtrackSep5-trainset95shuffle1/config.yaml
WtrackSep5-trainset95shuffle1,NeuroACT,=BLOB=,20,/media/labuser/NA_1_2025/spyglass/J16/position_models/WtrackSep5-trainset95shuffle1/config.yaml


In [7]:
proj_key = dict(
    project_name = project_name,
    config_path      = str(config_path),
    team_name = "NeuroACT",
    frames_per_video = 20,
    bodyparts = ["nose", "tailBase", "tailMid", "tailTip", "forepawL", "forepawR", "hindpawL", "hindpawR"]
)

sgp.DLCProject().insert1(proj_key, skip_duplicates=True)


In [19]:
sgp.DLCProject()

project_name name of DLC project,team_name,bodyparts list of bodyparts to label,frames_per_video number of frames to extract from each video,config_path path to config.yaml for model
wilbur-WtrackSep5-trainset95shuffle1,NeuroACT,=BLOB=,20,/media/labuser/NA_1_2025/spyglass/wilbur/position_models/WtrackSep5-trainset95shuffle1/config.yaml
WtrackSep5-trainset95shuffle1,NeuroACT,=BLOB=,20,/media/labuser/NA_1_2025/spyglass/J16/position_models/WtrackSep5-trainset95shuffle1/config.yaml


In [36]:
proj_row = (sgp.DLCProject & {"project_name": project_name}).fetch1()
proj_key = {
    "project_name": proj_row["project_name"],
    "config_path": proj_row["config_path"],
}
sgp.DLCModelInput.insert1(proj_key, skip_duplicates=True)


#### Insert into DLCProject.BodyPart()

In [ ]:
proj_key = (sgp.DLCProject & {"project_name": project_name}).fetch1("KEY")

bps = (sgp.DLCProject & proj_key).fetch1("bodyparts")
print(bps)
rows = [{"project_name": proj_key["project_name"], "bodypart": bp} for bp in bps]
print(rows)
sgp.DLCProject().BodyPart.insert(rows, skip_duplicates=True)

['nose', 'tailBase', 'tailMid', 'tailTip', 'forepawL', 'forepawR', 'hindpawL', 'hindpawR']
[{'project_name': 'wilbur-WtrackSep5-trainset95shuffle1', 'bodypart': 'nose'}, {'project_name': 'wilbur-WtrackSep5-trainset95shuffle1', 'bodypart': 'tailBase'}, {'project_name': 'wilbur-WtrackSep5-trainset95shuffle1', 'bodypart': 'tailMid'}, {'project_name': 'wilbur-WtrackSep5-trainset95shuffle1', 'bodypart': 'tailTip'}, {'project_name': 'wilbur-WtrackSep5-trainset95shuffle1', 'bodypart': 'forepawL'}, {'project_name': 'wilbur-WtrackSep5-trainset95shuffle1', 'bodypart': 'forepawR'}, {'project_name': 'wilbur-WtrackSep5-trainset95shuffle1', 'bodypart': 'hindpawL'}, {'project_name': 'wilbur-WtrackSep5-trainset95shuffle1', 'bodypart': 'hindpawR'}]


#### Insert into DLCModelParams()

In [11]:
with open(config_path, "r") as f:
    dlc_params = yaml.safe_load(f)

params_dict = {
    "params": dlc_params,
    "shuffle": 1,
    "trainingsetindex": 0,
    "model_prefix": "",
}
sgp.DLCModelParams.insert1({"dlc_model_params_name" :"WtrackSep5", "params" : params_dict}, skip_duplicates = True)

#### Insert into DLCModelSelection()

In [38]:
sgp.DLCModelSelection().insert1({
    "project_name": project_name,
    "dlc_model_name": "WtrackSep5-trainset95shuffle1model",
    "dlc_model_params_name": "WtrackSep5"},
                                skip_duplicates = True)
sgp.DLCModelSelection()

project_name name of DLC project,dlc_model_name User-friendly model name,dlc_model_params_name
wilbur-WtrackSep5-trainset95shuffle1,WtrackSep5-trainset95shuffle1model,WtrackSep5
WtrackSep5-trainset95shuffle1,WtrackSep5-trainset95shuffle1model,WtrackSep5


In [ ]:
model_key = (sgp.DLCModelSelection() & {"project_name": project_name}).fetch1("KEY")
sgp.DLCModel.populate(model_key)
